In [1]:
import Control.Applicative (liftA3)
import Control.Monad (replicateM)
import Control.Monad.Trans.State
import System.Random

data Die =    DieOne
            | DieTwo
            | DieThree
            | DieFour
            | DieFive
            | DieSix
            deriving (Eq, Show)
            
intToDie :: Int -> Die
intToDie n =
    case n of
    1 -> DieOne
    2 -> DieTwo
    3 -> DieThree
    4 -> DieFour
    5 -> DieFive
    6 -> DieSix
    -- Use this tactic _extremely_ sparingly.
    x -> error $ "intToDie got non 1-6 integer: " ++ show x
    
rollDie :: State StdGen Die
rollDie = state $ do
    (n, s) <- randomR (1, 6)
    return (intToDie n, s)
    
rollsToGetN :: Int -> StdGen -> Int
rollsToGetN n g = go 0 0 g
    where go :: Int -> Int -> StdGen -> Int
          go sum count gen
              | sum >= n = count
              | otherwise =
                  let (die, nextGen) = randomR (1, 6) gen
                  in go (sum + die) (count + 1) nextGen
            
rollsCountLogged :: Int -> StdGen -> (Int, [Die])
rollsCountLogged n g = go 0 (0,[]) g
    where go :: Int -> (Int, [Die]) -> StdGen -> (Int, [Die])
          go sum (count, dices) gen
            | sum >= n = (count, dices)
            | otherwise =
                let (die, nextGen) = randomR (1, 6) gen
                in go (sum + die) (count + 1, (intToDie die) : dices) nextGen

g = mkStdGen 0

rollsCountLogged 20 g

(5,[DieFive,DieOne,DieFour,DieSix,DieSix])

In [2]:
{-# LANGUAGE InstanceSigs #-}

newtype Moi s a = Moi { runMoi :: s -> (a, s) }

instance Functor (Moi s) where
    fmap :: (a -> b) -> Moi s a -> Moi s b
    fmap f (Moi g) = Moi moi
        where moi s = let (a, s') = g s
                      in (f a, s')
instance Applicative (Moi s) where
    pure :: a -> Moi s a
    pure a = Moi (\s -> (a,s))
    (<*>) :: Moi s (a -> b) -> Moi s a-> Moi s b
    (Moi mf) <*> (Moi ma) = Moi moi
        where moi s = let (f, s' ) = mf s
                          (a, s'') = ma s'
                      in (f a, s'')
                      
instance Monad (Moi s) where
    return = pure
    (>>=) :: Moi s a -> (a -> Moi s b) -> Moi s b
    (Moi ma) >>= g = Moi moi
        where moi s = let (a, s') = ma s
                      in (runMoi (g a)) s'


In [3]:
get :: Moi s s
get = Moi (\s -> (s,s))

put :: s -> Moi s ()
put s = Moi (\_ -> ((),s))

exec :: Moi s a -> s -> s
exec (Moi moi) s = snd $ moi s

eval :: Moi s a -> s -> a
eval (Moi moi) s = fst $ moi s

modify :: (s -> s) -> Moi s ()
modify f = Moi (\s -> ((), f s) )